In [ ]:
from pprint import pprint
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

from metropolis import Metropolis
import os

def get_global_index_mask(token):
    # Machine-generated panoptic masks can be retrieved for any image with
    meta, mask = met.get_panoptic_mask(token)

    # Read the semantic label for each pixel and reassign a global index for them
    category_mapping = {cat['token']: idx for idx, cat in enumerate(met.category)}
    global_index_mask = np.zeros_like(mask)

    for idx in np.unique(mask):
        token = meta['category_tokens'][idx]
        global_index = category_mapping[token] if token in category_mapping else 0
        global_index_mask[mask == idx] = global_index

    return global_index_mask

to_save_folder="/home/yuheng/data/metropolis/yuheng_selected"

# Loading the dataset
met = Metropolis(
    "val",       # Name of the split we want to load (i.e. 'train', 'test', 'val')
    "/home/yuheng/data/metropolis/"  # Path to the dataset's root folder
)
# Create subfolders for each scene's token
for scene in met.scene:
    scene_folder = os.path.join(to_save_folder, scene["token"])
    os.makedirs(scene_folder, exist_ok=True)
    # Create subsubfolder for images
    images_folder = os.path.join(scene_folder, "images")
    os.makedirs(images_folder, exist_ok=True)
    
    pose_save_path = os.path.join(scene_folder, "ego_poses.txt")
    if os.path.exists(pose_save_path):
        os.remove(pose_save_path)

    category_index_mapping = {idx: cat['description'] for idx, cat in enumerate(met.category)}
    with open(os.path.join(to_save_folder, "semantic_info.txt"), 'w') as f:
        for idx, cat in category_index_mapping.items():
            f.write(f"{idx}:{cat}\n")

    # Save the CAM_FRONT images
    sample_token = scene["first_sample_token"]
    index = 0
    while sample_token:
        sample = met.get("sample", sample_token)
        sample_token = sample["next_sample"]
        
        if not "CAM_FRONT" in sample["data"]:
            print(f"Skipping sample {sample_token} as it does not have a CAM_FRONT image")
            
            if os.path.exists(images_folder):
                os.rmdir(images_folder)
            if os.path.exists(scene_folder):
                os.rmdir(scene_folder)
            continue
            
        cam_front_data = met.get("sample_data", sample["data"]["CAM_FRONT"])
        image_path = met.get_sample_data_path(cam_front_data["token"])
        
        image = Image.open(image_path)
        image_save_path = os.path.join(images_folder, f"{index:06d}.png")
        image.save(image_save_path)
        
        # Save the depth image
        depth_folder = os.path.join(scene_folder, "depth")
        os.makedirs(depth_folder, exist_ok=True)
        
        depth_data = met.get("sample_data", sample["data"]["DEPTH_FRONT"])
        depth_image_path = met.get_sample_data_path(depth_data["token"])
        
        depth_image = Image.open(depth_image_path)
        depth_image_save_path = os.path.join(depth_folder, f"{index:06d}.png")
        depth_image.save(depth_image_save_path)

        # Save the semantic information
        semantic_folder = os.path.join(scene_folder, "semantic")
        os.makedirs(semantic_folder, exist_ok=True)
        
        semantic_mask = get_global_index_mask(sample["data"]["CAM_FRONT"])
        semantic_save_path = os.path.join(semantic_folder, f"{index:06d}.npy")
        np.save(semantic_save_path, semantic_mask)
        
        # Save the ego pose information
        

        camera = met.get("sample_data", sample["data"]["CAM_FRONT"])
        ego_pose = met.get("ego_pose", camera["ego_pose_token"])

        with open(pose_save_path, 'a') as f:
            rotation_str = ' '.join(map(str, ego_pose['rotation']))
            translation_str = ' '.join(map(str, ego_pose['translation']))
            f.write(f"{index} {rotation_str} {translation_str}\n")

        index += 1


Loading Metropolis tables for split val...
88 category,
13 attribute,
40919 instance,
12 sensor,
12 calibrated_sensor,
3429 ego_pose,
4580 panoptic,
48 scene,
4580 sample,
42283 sample_data,
80452 sample_annotation,
349936 sample_annotation_2d,
Done loading in 1.226 seconds.
Reverse indexing ...
Done reverse indexing in 0.7 seconds.
Skipping sample KiTvdtcskQ5Q3Zj39aaHcw as it does not have a CAM_FRONT image
Skipping sample f8cawu8qZx4e6h4uT3sShg as it does not have a CAM_FRONT image
Skipping sample d5UfqgL-dZ8_Ih3KX3AOpQ as it does not have a CAM_FRONT image
Skipping sample kbH89gqS4lYHrKPUsiBhAQ as it does not have a CAM_FRONT image
Skipping sample sIxhS_ZcV8MCOsDCi1N0Jg as it does not have a CAM_FRONT image
Skipping sample tR50hpeZ_QH3K_EuXDx0kw as it does not have a CAM_FRONT image
Skipping sample qq77bzXhCd9Z2ZtPBvXRUw as it does not have a CAM_FRONT image
Skipping sample Ei4G4iMRNkuCdJifKP4xnw as it does not have a CAM_FRONT image
Skipping sample 6BYgT7WMYI_6Ih6ODIIxaw as it does